In [3]:
import argparse
import os
import logging
import torch
import numpy as np

from tqdm import tqdm
from easydict import EasyDict

from models import build_model
from data import DataManger_Epoch, DataManger_Episode
from logger import setup_logging
from utils import read_config, rmdir, summary, array_interweave, COLOR
from evaluators import recognition_metrics

In [4]:
config1 = "config/baseline_peta.yml"
config2 = "config/episode_peta.yml"

resume1 = "/content/drive/Shared drives/REID/HIEN/Models/OSNet_Person_Attribute_Refactor/checkpoints/0731_232453/model_best_accuracy.pth"
resume2 = "/content/drive/Shared drives/REID/HIEN/Models/person_attribute_recognition/checkpoints/0809_231322/model_best_accuracy.pth"

config1 = read_config(config1)
config1.update({'resume': resume1})
config1.update({'colab': True})

config2 = read_config(config2)
config2.update({'resume': resume2})
config2.update({'colab': True})

datamanager1 = DataManger_Epoch(config1['data'])
datamanager2 = DataManger_Episode(config2['data'])

weight = datamanager1.datasource.get_weight('train')

Downloading...
Downloaded!
Downloading...
Downloaded!


In [9]:
from torch.distributions.multinomial import Multinomial
m = Multinomial(32, torch.exp(1-torch.tensor(weight)))

In [10]:
attribute_name = datamanager1.datasource.get_attribute()

In [11]:
import random
selected_attribute = random.sample(attribute_name, 8)
print(selected_attribute)

['carryingNothing', 'lowerBodyFormal', 'personalLess60', 'upperBodyVNeck', 'lowerBodyTrousers', 'accessoryMuffler', 'upperBodyCasual', 'upperBodyFormal']


In [13]:
import torch
torch.multinomial(torch.exp(1-torch.tensor(weight)), 8, replacement=True)

tensor([25, 14,  9, 14, 23, 28, 27, 29])

In [ ]:
import collections
from collections import defaultdict
result = defaultdict(int)
for _ in range(100000):
    temp = torch.multinomial(torch.exp(1-torch.tensor(weight)), 8, replacement=True)
    for key, value in collections.Counter(temp.numpy()).items():
        result[key] += value
print(list(collections.OrderedDict(sorted(result.items())).values()))

In [6]:
import numpy as np
temp = np.random.multinomial(8, np.exp(1-np.array(weight)))
temp

ValueError: pvals < 0, pvals > 1 or pvals contains NaNs